<a href="https://colab.research.google.com/github/shshahul/Student_database_MongoDB/blob/main/Task2MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo
from pymongo import MongoClient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
client = MongoClient("enter DB credentials")

In [ ]:
client.list_database_names()

['CRUD_task',
 'Task2_MongoDB',
 'Tweets_database',
 'marks',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'shahul_db',
 'admin',
 'local']

#methods of uploading JSON data
1)Mondo db Compass we can directly upload by creating the file 
2)Another method is by using python 

In [ ]:
db = client['marks']
db.list_collection_names()

['students_data']

In [ ]:
my_col=db['students_data']

In [ ]:
from pprint import pprint
for i in my_col.find():
  print(i)
  

1.Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [ ]:
#Find the student name who scored maximum scores in all (exam, quiz and homework)?
#stage 2 we need to submit (exam, quiz and homework) one by one
stage1={"$unwind":"$scores"}
stage2={"$match":{"scores.type":"homework"}}    
stage3={"$sort":{"scores.score":-1}}
stage4={"$limit":1}
stage5={"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
    pprint(i)


{'name': 'Whitley Fears',
 'scores': {'score': 99.77237745070993, 'type': 'homework'}}


2)Find students who scored below average in the exam and pass mark is 40%?

In [ ]:
stage1 = {"$unwind":"$scores"}
stage2={"$match":{"scores.type":"exam"}}
stage3 = {"$group": {"_id":"$scores.type", "avg_score":{"$avg":"$scores.score"}}}
for i in my_col.aggregate([stage1,stage2,stage3]):
    pprint(i)  


{'_id': 'exam', 'avg_score': 48.67367075950175}


In [ ]:
stage1 = {"$unwind":"$scores"}
stage2 = {"$group": {"_id":"$scores.type", "avg_score":{"$avg":"$scores.score"}}}
for i in my_col.aggregate([stage1,stage2]):
    pprint(i)

{'_id': 'exam', 'avg_score': 48.67367075950175}
{'_id': 'homework', 'avg_score': 67.81869620661149}
{'_id': 'quiz', 'avg_score': 48.99672319430254}


a)Students scored below 40%.
#in the above cell I calculated the overall average score for ["quiz",'exam','homework']

In [ ]:
#Students scored below 40%
stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$lt":40},"scores.type":"exam"}}
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}
for i in my_col.aggregate([stage1,stage2,stage3]):
  print(i)

{'name': 'aimee Zank', 'scores': {'score': 1.463179736705023, 'type': 'exam'}}
{'name': 'Jenette Flanders', 'scores': {'score': 37.32285459166097, 'type': 'exam'}}
{'name': 'Daphne Zheng', 'scores': {'score': 22.13583712862635, 'type': 'exam'}}
{'name': 'Alix Sherrill', 'scores': {'score': 25.15924151998215, 'type': 'exam'}}
{'name': 'Dodie Staller', 'scores': {'score': 7.772386442858281, 'type': 'exam'}}
{'name': 'Fletcher Mcconnell', 'scores': {'score': 39.41011069729274, 'type': 'exam'}}
{'name': 'Jonie Raby', 'scores': {'score': 19.17861192576963, 'type': 'exam'}}
{'name': 'Edgar Sarkis', 'scores': {'score': 8.606983261043888, 'type': 'exam'}}
{'name': 'Laureen Salomone', 'scores': {'score': 3.677565278992456, 'type': 'exam'}}
{'name': 'Kaila Deibler', 'scores': {'score': 15.89771199662455, 'type': 'exam'}}
{'name': 'Tandra Meadows', 'scores': {'score': 24.90138146001744, 'type': 'exam'}}
{'name': 'Carli Belvins', 'scores': {'score': 7.112266875518214, 'type': 'exam'}}
{'name': 'Sy

3)Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [ ]:
#I am considering below 40 marks as faile and i am adding new column ["Exam_result"] and assigned them as fail
stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$lt":40}}}  
stage4 = {"$addFields":{'Exam_Result':'Fail'}}
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}

for i in my_col.aggregate([stage1,stage2,stage3,stage4]):

    pprint(i) 

In [ ]:
# I am considering above 40 marks as faile and i am adding new column ["Exam_result"] and assigned them as pass
stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$gte":40}}}  
stage4 = {"$addFields":{'Exam_Result':'Pass'}}
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4]):

    pprint(i)


4)       Find the total and average of the exam, quiz and homework and store them in a separate collection.

In [ ]:
#finding the average of the exam, quiz and homework and store them in a separate collection.
Average =[]
stage1 = {"$unwind":"$scores"}
stage2 = {"$group": {"_id":"$scores.type", "avg_score":{"$avg":"$scores.score"}}}
for i in my_col.aggregate([stage1,stage2]):
  Average.append(i)
print(Average)


[{'_id': 'quiz', 'avg_score': 48.99672319430254}, {'_id': 'homework', 'avg_score': 67.81869620661149}, {'_id': 'exam', 'avg_score': 48.67367075950175}]


In [ ]:
Avg_3scores_col = db.Total_AVG_Score_col
Avg_3scores_col.insert_many(Average)

In [ ]:
##finding  total score of the exam, quiz and homework and store them in a separate collection.

Total_score =[]
stage1 = {"$unwind":"$scores"}
stage2 = {"$group": {"_id":"$scores.type", "total_score":{"$sum":"$scores.score"}}}
for i in my_col.aggregate([stage1,stage2]):
  Total_score.append(i)
print(Total_score)


[{'_id': 'quiz', 'total_score': 9799.344638860508}, {'_id': 'exam', 'total_score': 9734.734151900351}, {'_id': 'homework', 'total_score': 13563.739241322297}]


In [ ]:
#Updating in the separate collection
total_score_col = db.Total_Scores_collection
total_score_col.insert_many(Total_score)


5)      Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [ ]:
data = []

stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$gte":40,"$lt":48}}}  
stage4 = {"$addFields":{'Exam_Result':'Pass'}}
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4]):
  data.append(i)
print(data)
   

In [ ]:
#creating new collection and inserting this collected data
db = client['marks']
avg_col = db.avg_student_col_marks
avg_col.insert_many(data)

6)      Create a new collection which consists of students who scored below the fail mark in all the categories.

In [ ]:
Fail_Marks = []

#I am considering below 40 marks as faile and i am adding new column ["Exam_result"] and assigned them as fail
stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$lt":40}}}  
stage4 = {"$addFields":{'Exam_Result':'Fail'}}
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}

for i in my_col.aggregate([stage1,stage2,stage3,stage4]):
  #pprint(i)
  Fail_Marks.append(i)
  print(Fail_Marks)






In [ ]:
fail_mark_col = db.Fail_students_catogery_col
fail_mark_col.insert_many(Fail_Marks)

In [ ]:
#To check the data inserted or not 
fail_mark_col.find_one()

{'_id': ObjectId('63d51e99e732936a798294bc'),
 'name': 'aimee Zank',
 'scores': {'score': 1.463179736705023, 'type': 'exam'},
 'Exam_Result': 'Fail'}

7)      Create a new collection which consists of students who scored above pass mark in all the categories.

In [ ]:
Pass_Students = []

stage1 = {"$unwind":"$scores"}
stage2 = {"$match":{"scores.score":{"$gte":40}}}  
stage3 = {"$project":{"_id":0,"name":1,"scores.score":1,"scores.type":1}}
stage4 = {"$addFields":{'Exam_Result':'Pass'}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4]):
  # pprint(i)

  Pass_Students.append(i)
print(Pass_Students)





In [ ]:
#Updating in separate collection
Pass_Students_col = db.Pass_Students_collection

Pass_Students_col.insert_many(Pass_Students)

In [ ]:
Pass_Students_col.find_one()

{'_id': ObjectId('63d5215ae732936a79829580'),
 'name': 'Aurelia Menendez',
 'scores': {'score': 60.06045071030959, 'type': 'exam'},
 'Exam_Result': 'Pass'}